In [1]:
import exmp
import qiime2
import tempfile
import os.path
import pandas as pd
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.taxa.methods import collapse

taxonomy = exmp.load_taxonomy()
sample_metadata = exmp.load_sample_metadata()
data_dir = exmp.cm_path

rarefied_table = qiime2.Artifact.load(os.path.join(data_dir, "rarefied_table.qza"))
uu_dm = qiime2.Artifact.load(os.path.join(data_dir, "unweighted_unifrac_distance_matrix.qza"))
wu_dm = qiime2.Artifact.load(os.path.join(data_dir, "weighted_unifrac_distance_matrix.qza"))
faith_pd = qiime2.Artifact.load(os.path.join(data_dir, "faith_pd_vector.qza"))
shannon = qiime2.Artifact.load(os.path.join(data_dir, "shannon_vector.qza"))
evenness = qiime2.Artifact.load(os.path.join(data_dir, "evenness_vector.qza"))

In [2]:
with tempfile.TemporaryDirectory() as output_dir:
    _, _, _, sample_metadata = exmp.ols_and_anova('RER_change', 'exmp1', '1.0', 
                                                  output_dir, 'week',
                                                  sample_metadata, uu_dm, wu_dm, 
                                                  faith_pd, shannon, evenness)

/Users/gregcaporaso/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.08203129307423626 and the largest is 2.6177950743186975.
  RuntimeWarning
/Users/gregcaporaso/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [3]:
rarefied_table = filter_samples(table=rarefied_table, metadata=sample_metadata).filtered_table
taxa_table = collapse(table=rarefied_table, taxonomy=taxonomy, level=6).collapsed_table.view(pd.DataFrame)

In [4]:
sample_metadata = sample_metadata.to_dataframe()

In [14]:
sorted_wu_pc3_correlations = pd.DataFrame(taxa_table.corrwith(sample_metadata['Weighted_UniFrac_PC3'], method='spearman').sort_values(), columns=['Spearman rho'])
sorted_wu_pc3_correlations['25th percentile rarefied count'] = taxa_table[sorted_wu_pc3_correlations.index].quantile(0.25)
sorted_wu_pc3_correlations['Median rarefied count'] = taxa_table[sorted_wu_pc3_correlations.index].quantile(0.50)
sorted_wu_pc3_correlations['75th percentile rarefied count'] = taxa_table[sorted_wu_pc3_correlations.index].quantile(0.75)
sorted_uu_pc3_correlations = pd.DataFrame(taxa_table.corrwith(sample_metadata['Unweighted_UniFrac_PC3'], method='spearman').sort_values(), columns=['Spearman rho'])
sorted_uu_pc3_correlations['25th percentile rarefied count'] = taxa_table[sorted_uu_pc3_correlations.index].quantile(0.25)
sorted_uu_pc3_correlations['Median rarefied count'] = taxa_table[sorted_uu_pc3_correlations.index].quantile(0.50)
sorted_uu_pc3_correlations['75th percentile rarefied count'] = taxa_table[sorted_uu_pc3_correlations.index].quantile(0.75)

The data are most easily interpreted if the ordination axes are positively correlated with the RER change. Since the direction of the PCs are arbitrary, I generally just run this a few times till I get a positive correlation.

In [15]:
sample_metadata['Weighted_UniFrac_PC3'].corr(sample_metadata['RER_change'])

0.21917073562928502

In [16]:
sample_metadata['Unweighted_UniFrac_PC3'].corr(sample_metadata['RER_change'])

0.23468802515225493

In [17]:
sample_metadata['shannon'].corr(sample_metadata['RER_change'])

0.31676905444457676

In [18]:
sample_metadata['pielou_e'].corr(sample_metadata['RER_change'])

0.2516514998264882

In [19]:
output_dir = os.path.join(exmp.cm_path, 'ols-and-anova', 'exmp1-RER_change-week1.0')
sorted_wu_pc3_correlations.to_csv(open(os.path.join(output_dir, 'wu-pcoa3-genus-correlations.csv'), 'w'))
sorted_uu_pc3_correlations.to_csv(open(os.path.join(output_dir, 'uu-pcoa3-genus-correlations.csv'), 'w'))